In [1]:
import pandas as pd 
import json
df_pipe = pd.read_excel('/Users/vitol/vscode/firm_perf/firm_perf_code/最终数据/Stk_info_industry_total_cleaned.xlsx',index_col=False)
df_pipe.drop(columns=['Unnamed: 0'],inplace=True)
df_pipe = df_pipe.rename(columns = {'Stkcd_old':'Stkcd'})
df_mv = pd.read_excel('/Users/vitol/vscode/firm_perf/firm_perf_code/最终数据/总市值.xlsx')
df_mv.rename(columns = {'证券代码':'Stkcd'},inplace=True)
df_mv.head()

,Stkcd,证券简称,总市值
0,000001.SZ,平安银行,2.190928e+11
1,000002.SZ,万科A,6.670799e+10
2,000004.SZ,*ST国华,1.129204e+09
3,000006.SZ,深振业A,8.761468e+09
4,000007.SZ,全新好,2.525606e+09


In [2]:
df_mv = pd.merge(df_mv, df_pipe, on=['Stkcd'],how = 'left')
df_basic_info = pd.read_excel('/Users/vitol/vscode/firm_perf/firm_perf_code/最终数据/股票基本信息（含行业）drop后.xlsx')[['stk_code','Stknme','公司全称']]
df_basic_info = df_basic_info.rename(columns = {'stk_code':'Stkcd_filled'})
df_mv = pd.merge(df_mv, df_basic_info, on=['Stkcd_filled'],how = 'left')
df_mv

,Stkcd,证券简称,总市值,申万行业分类,Stkcd_filled,Stknme,公司全称
0,000001.SZ,平安银行,2.190928e+11,银行,1.0,平安银行,平安银行股份有限公司
1,000002.SZ,万科A,6.670799e+10,房地产,2.0,万科A,万科企业股份有限公司
2,000004.SZ,*ST国华,1.129204e+09,计算机,4.0,*ST 国华,深圳国华网安科技股份有限公司
3,000006.SZ,深振业A,8.761468e+09,房地产,6.0,深振业A,深圳市振业(集团)股份有限公司
4,000007.SZ,全新好,2.525606e+09,商贸零售,7.0,全新好,深圳市全新好股份有限公司
...,...,...,...,...,...,...,...
5489,920445.BJ,龙竹科技,1.586844e+09,轻工制造,920445.0,龙竹科技,龙竹科技集团股份有限公司
5490,920489.BJ,佳先股份,3.298219e+09,基础化工,920489.0,佳先股份,安徽佳先功能助剂股份有限公司
5491,920682.BJ,球冠电缆,3.585504e+09,电力设备,920682.0,球冠电缆,宁波球冠电缆股份有限公司
5492,920799.BJ,艾融软件,1.262247e+10,计算机,920799.0,艾融软件,上海艾融软件股份有限公司


In [3]:
df_mv['id'] = df_mv.apply(lambda row: f"{row['Stkcd']} {row['Stknme']} {row['公司全称']}", axis=1)
df_mv = df_mv.drop(columns=['Stkcd','证券简称','Stkcd_filled','Stknme', '公司全称'])
df_mv

,总市值,申万行业分类,id
0,2.190928e+11,银行,000001.SZ 平安银行 平安银行股份有限公司
1,6.670799e+10,房地产,000002.SZ 万科A 万科企业股份有限公司
2,1.129204e+09,计算机,000004.SZ *ST 国华 深圳国华网安科技股份有限公司
3,8.761468e+09,房地产,000006.SZ 深振业A 深圳市振业(集团)股份有限公司
4,2.525606e+09,商贸零售,000007.SZ 全新好 深圳市全新好股份有限公司
...,...,...,...
5489,1.586844e+09,轻工制造,920445.BJ 龙竹科技 龙竹科技集团股份有限公司
5490,3.298219e+09,基础化工,920489.BJ 佳先股份 安徽佳先功能助剂股份有限公司
5491,3.585504e+09,电力设备,920682.BJ 球冠电缆 宁波球冠电缆股份有限公司
5492,1.262247e+10,计算机,920799.BJ 艾融软件 上海艾融软件股份有限公司


In [4]:
df_mv['公司总市值规模在所在行业排在前%'] = df_mv.groupby('申万行业分类')['总市值'].rank(pct=True)
df_mv['公司总市值规模在所在行业排在前%'] = df_mv['公司总市值规模在所在行业排在前%'].apply(lambda x: round(x*100, 2))
df_mv = df_mv.drop(columns=['申万行业分类'])
df_mv

,总市值,id,公司总市值规模在所在行业排在前%
0,2.190928e+11,000001.SZ 平安银行 平安银行股份有限公司,78.57
1,6.670799e+10,000002.SZ 万科A 万科企业股份有限公司,98.20
2,1.129204e+09,000004.SZ *ST 国华 深圳国华网安科技股份有限公司,0.55
3,8.761468e+09,000006.SZ 深振业A 深圳市振业(集团)股份有限公司,66.67
4,2.525606e+09,000007.SZ 全新好 深圳市全新好股份有限公司,12.87
...,...,...,...
5489,1.586844e+09,920445.BJ 龙竹科技 龙竹科技集团股份有限公司,3.03
5490,3.298219e+09,920489.BJ 佳先股份 安徽佳先功能助剂股份有限公司,32.10
5491,3.585504e+09,920682.BJ 球冠电缆 宁波球冠电缆股份有限公司,28.65
5492,1.262247e+10,920799.BJ 艾融软件 上海艾融软件股份有限公司,79.83


In [ ]:
json_list = []
col = df_mv.columns.tolist()
def trans(sth):
    content = {}
    for i in col:
        if i !='id':
            content[i] = sth[i]
    text = {
        'text': content,
        "id": sth['id']
    }

    json_list.append(text)
df_mv.apply(lambda row:trans(row), axis=1)
json_list


[{'text': {'总市值': 219092816455.42, '公司总市值规模在所在行业排在前%': 78.57},
  'id': '000001.SZ 平安银行 平安银行股份有限公司'},
 {'text': {'总市值': 66707988216.38, '公司总市值规模在所在行业排在前%': 98.2},
  'id': '000002.SZ 万科A 万科企业股份有限公司'},
 {'text': {'总市值': 1129203805.46, '公司总市值规模在所在行业排在前%': 0.55},
  'id': '000004.SZ *ST 国华 深圳国华网安科技股份有限公司'},
 {'text': {'总市值': 8761467848.54, '公司总市值规模在所在行业排在前%': 66.67},
  'id': '000006.SZ 深振业A 深圳市振业(集团)股份有限公司'},
 {'text': {'总市值': 2525606240.76, '公司总市值规模在所在行业排在前%': 12.87},
  'id': '000007.SZ 全新好 深圳市全新好股份有限公司'},
 {'text': {'总市值': 7850331503.87, '公司总市值规模在所在行业排在前%': 77.33},
  'id': '000008.SZ 神州高铁 神州高铁技术股份有限公司'},
 {'text': {'总市值': 21046385954.4, '公司总市值规模在所在行业排在前%': 85.94},
  'id': '000009.SZ 中国宝安 中国宝安集团股份有限公司'},
 {'text': {'总市值': 3288005555.12, '公司总市值规模在所在行业排在前%': 44.51},
  'id': '000010.SZ 美丽生态 深圳美丽生态股份有限公司'},
 {'text': {'总市值': 4587356239.89436, '公司总市值规模在所在行业排在前%': 39.19},
  'id': '000011.SZ 深物业A 深圳市物业发展(集团)股份有限公司'},
 {'text': {'总市值': 11229073146.9006, '公司总市值规模在所在行业排在前%': 82.24},
  'id': '000012.S

In [6]:
# 指定要保存的文件名
filename = 'stk_mv.json'

# 使用 with 语句打开文件，并使用 json.dump 方法将 json_list 写入文件
with open(filename, 'w', encoding='utf-8') as f:
    # 如果 json_list 是一个列表或字典，直接使用 dump
    json.dump(json_list, f, ensure_ascii=False, indent=4)

print(f"数据已保存到 {filename}")

数据已保存到 stk_mv.json
